In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mlflow
import numpy as np
from sklearn.metrics import mean_squared_error
from lib.experiment_setup import Experiment

ModuleNotFoundError: No module named 'experiments'

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

experiment = Experiment(name="GBR", norm=3)
models = []
rmse_scores = []

gbr_params = {
    'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1.0],
    'criterion': ['friedman_mse', 'squared_error'],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'min_weight_fraction_leaf': [0.0, 0.1],
    'max_depth': [3, 4, 5],
    'min_impurity_decrease': [0.0, 0.01],
    'init': [None, 'zero'],
    'random_state': [None, 42],
    'max_features': [None, 'sqrt', 'log2'],
    'alpha': [0.9, 0.95],
    'verbose': [0],
    'max_leaf_nodes': [None, 4, 8],
    'warm_start': [False, True],
    'validation_fraction': [0.1],
    'n_iter_no_change': [None, 5],
    'tol': [1e-4],
    'ccp_alpha': [0.0, 0.01]
}

gb_regressor = GradientBoostingRegressor()
rand_search = RandomizedSearchCV(estimator=gb_regressor, param_distributions=gbr_params, n_iter=150, cv=5, n_jobs=-1, verbose=3)


def run_experiment(X_train, X_test, y_train, y_test, target, norm):
    rand_search.fit(X_train, y_train[target])
    best_estimator = rand_search.best_estimator_

    # Predict using the best estimator
    y_pred = best_estimator.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test[target], y_pred))

    # MLflow logging
    mlflow.log_metric("rmse", float(rmse))

    # Log best parameters from the grid search
    mlflow.log_params(rand_search.best_params_)
    mlflow.log_param("target", target)
    mlflow.log_param("norm", norm)

    # Log the best estimator as a model
    mlflow.sklearn.log_model(best_estimator, f"model_{target}_{norm}")


experiment.run(run_experiment)

2024/03/11 11:25:07 INFO mlflow.tracking.fluent: Experiment with name 'GBR_Norm3_20240311-112506' does not exist. Creating a new experiment.


Starting training for target: SiO2
Fitting 5 folds for each of 150 candidates, totalling 750 fits


KeyboardInterrupt: 